# Titanic Survival Analysis
- The Titanic dataset is a well-known dataset used for classification tasks, particularly for predicting survival outcomes. It contains information on passengers aboard the Titanic when it sank in 1912.
### Dataset Columns
- The Titanic dataset typically includes the following columns:
1. PassengerId – Unique identifier for each passenger
2. Survived – Survival status (0 = No, 1 = Yes)
3. Pclass – Ticket class (1 = First, 2 = Second, 3 = Third)
4. Name – Passenger’s full name
5. Sex – Gender (male/female)
6. Age – Age of the passenger
7. SibSp – Number of siblings/spouses aboard
8. Parch – Number of parents/children aboard
9. Ticket – Ticket number
10. Fare – Passenger fare
11. Cabin – Cabin number (often missing)
12. Embarked – Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

## Data Preprocessing and Cleaning

In [2]:
import os
import mysql.connector  # Use pymysql or psycopg2 for PostgreSQL
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Fetch credentials
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT")

# Connect to MySQL Database
try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        port=int(DB_PORT)
    )
    cursor = conn.cursor()

    # Execute SQL Query
    cursor.execute("SELECT * FROM titanicdata LIMIT 10;")  # Adjust table name
    rows = cursor.fetchall()

    # Print fetched data
    for row in rows:
        print(row)

    # Close connections
    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error: {e}")

Error: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
